# Slack

This notebook walks through connecting LangChain to your `Slack` account.

To use this toolkit, you will need to get a token explained in the [Slack API docs](https://api.slack.com/tutorials/tracks/getting-a-token). Once you've received a SLACK_USER_TOKEN, you can input it as an environmental variable below.

In [ ]:
!pip install --upgrade slack_sdk > /dev/null
!pip install beautifulsoup4 > /dev/null # This is optional but is useful for parsing HTML messages

## Assign Environmental Variables

The toolkit will read the SLACK_USER_TOKEN environmental variable to authenticate the user so you need to set them here. You will also need to set your OPENAI_API_KEY to use the agent later.

In [ ]:
# Set environmental variables here

## Create the Toolkit and Get Tools

To start, you need to create the toolkit, so you can access its tools later.

In [ ]:
from langchain.agents.agent_toolkits import SlackToolkit

toolkit = SlackToolkit()
tools = toolkit.get_tools()
tools

## Use within an Agent

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    verbose=False,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent.run("Send a greeting to my coworkers in the #general channel.")

In [ ]:
agent.run("How many channels are in the workspace? Please list out their names.")

In [ ]:
agent.run(
    "Tell me the number of messages sent in the #introductions channel from the past month."
)